# Running models and cross validation 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn import metrics

In [3]:
#Time series split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [4]:
df = pd.read_csv('')

FileNotFoundError: [Errno 2] No such file or directory: ''

In [5]:
# Separating features and target variable
X = df(['Price'], axis=1)
y = df['Price']

NameError: name 'df' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Cross validation

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

Iterate through cross validation

In [ ]:
for train_index, test_index in tscv.split(data):
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

Create the dataframe 

In [9]:
df1 = pd.read_csv('merged_luisfer.CSV')
df2 = pd.read_csv('merged_isabella.csv')
df3 = pd.read_csv('merged_camilo.csv', index_col=False)

In [10]:
df1

,Unnamed: 0,Date,Solar,Wind Onshore,country_index,Wind Offshore
0,0,2015-01-01 00:00:00+01:00,0.0,118.0,AT,NaN
1,1,2015-01-01 00:15:00+01:00,0.0,118.0,AT,NaN
2,2,2015-01-01 00:30:00+01:00,0.0,117.0,AT,NaN
3,3,2015-01-01 00:45:00+01:00,0.0,116.0,AT,NaN
4,4,2015-01-01 01:00:00+01:00,0.0,146.0,AT,NaN
...,...,...,...,...,...,...
705207,78611,2023-12-30 19:00:00+01:00,0.0,665.0,DK_1,570.0
705208,78612,2023-12-30 20:00:00+01:00,0.0,466.0,DK_1,452.0
705209,78613,2023-12-30 21:00:00+01:00,0.0,323.0,DK_1,352.0
705210,78614,2023-12-30 22:00:00+01:00,0.0,227.0,DK_1,315.0


In [11]:
df2

,Unnamed: 0,Date,Solar,Wind Onshore,country_index,Wind Offshore
0,0,2015-01-01 00:00:00+01:00,0.0,NaN,IT_SARD,NaN
1,1,2015-01-01 01:00:00+01:00,0.0,NaN,IT_SARD,NaN
2,2,2015-01-01 02:00:00+01:00,0.0,NaN,IT_SARD,NaN
3,3,2015-01-01 03:00:00+01:00,0.0,NaN,IT_SARD,NaN
4,4,2015-01-01 04:00:00+01:00,0.0,NaN,IT_SARD,NaN
...,...,...,...,...,...,...
1048570,76801,2023-10-08 16:00:00+02:00,0.0,89.0,NO_4,NaN
1048571,76802,2023-10-08 17:00:00+02:00,0.0,83.0,NO_4,NaN
1048572,76803,2023-10-08 18:00:00+02:00,0.0,76.0,NO_4,NaN
1048573,76804,2023-10-08 19:00:00+02:00,0.0,74.0,NO_4,NaN


In [12]:
df3

,Unnamed: 0,Date,Solar,Wind Offshore,country_index,Wind Onshore
0,0,2015-01-07 00:00:00+01:00,0.0,316.0,DK_2,267.0
1,1,2015-01-07 01:00:00+01:00,0.0,302.0,DK_2,235.0
2,2,2015-01-07 02:00:00+01:00,0.0,269.0,DK_2,204.0
3,3,2015-01-07 03:00:00+01:00,0.0,234.0,DK_2,173.0
4,4,2015-01-07 04:00:00+01:00,0.0,182.0,DK_2,142.0
...,...,...,...,...,...,...
1048570,58440,2021-09-01 07:00:00+02:00,65.0,NaN,IT_CNOR,3.0
1048571,58441,2021-09-01 08:00:00+02:00,271.0,NaN,IT_CNOR,4.0
1048572,58442,2021-09-01 09:00:00+02:00,509.0,NaN,IT_CNOR,4.0
1048573,58443,2021-09-01 10:00:00+02:00,684.0,NaN,IT_CNOR,6.0


In [13]:
print(df1.shape)
print(df2.shape)
print(df3.shape)

(705212, 6)
(1048575, 6)
(1048575, 6)


In [14]:
df_concatenated = pd.concat([df1, df2, df3], ignore_index=True)

In [15]:
df_concatenated

,Unnamed: 0,Date,Solar,Wind Onshore,country_index,Wind Offshore
0,0,2015-01-01 00:00:00+01:00,0.0,118.0,AT,NaN
1,1,2015-01-01 00:15:00+01:00,0.0,118.0,AT,NaN
2,2,2015-01-01 00:30:00+01:00,0.0,117.0,AT,NaN
3,3,2015-01-01 00:45:00+01:00,0.0,116.0,AT,NaN
4,4,2015-01-01 01:00:00+01:00,0.0,146.0,AT,NaN
...,...,...,...,...,...,...
2802357,58440,2021-09-01 07:00:00+02:00,65.0,3.0,IT_CNOR,NaN
2802358,58441,2021-09-01 08:00:00+02:00,271.0,4.0,IT_CNOR,NaN
2802359,58442,2021-09-01 09:00:00+02:00,509.0,4.0,IT_CNOR,NaN
2802360,58443,2021-09-01 10:00:00+02:00,684.0,6.0,IT_CNOR,NaN


In [16]:
print(df_concatenated.head())


   Unnamed: 0                       Date  Solar  Wind Onshore country_index  \
0           0  2015-01-01 00:00:00+01:00    0.0         118.0            AT   
1           1  2015-01-01 00:15:00+01:00    0.0         118.0            AT   
2           2  2015-01-01 00:30:00+01:00    0.0         117.0            AT   
3           3  2015-01-01 00:45:00+01:00    0.0         116.0            AT   
4           4  2015-01-01 01:00:00+01:00    0.0         146.0            AT   

   Wind Offshore  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  


In [17]:
print(df_concatenated.tail()) 

         Unnamed: 0                       Date  Solar  Wind Onshore  \
2802357       58440  2021-09-01 07:00:00+02:00   65.0           3.0   
2802358       58441  2021-09-01 08:00:00+02:00  271.0           4.0   
2802359       58442  2021-09-01 09:00:00+02:00  509.0           4.0   
2802360       58443  2021-09-01 10:00:00+02:00  684.0           6.0   
2802361       58444  2021-09-01 11:00:00+02:00  784.0           3.0   

        country_index  Wind Offshore  
2802357       IT_CNOR            NaN  
2802358       IT_CNOR            NaN  
2802359       IT_CNOR            NaN  
2802360       IT_CNOR            NaN  
2802361       IT_CNOR            NaN  


In [18]:
df_concatenated.to_csv('all_merged.csv', index=False)

Now we're missing merging all the dataframe


In [120]:
df4 = pd.read_csv('all_merged.csv')

In [121]:
df4['Date'] = pd.to_datetime(df4['Date'])
df4.set_index(['Date'], inplace=True)
df4.index = pd.PeriodIndex(df4.index.get_level_values(0), freq='H')

In [122]:
df4 = df4.reset_index()

In [123]:
df4.set_index(['Date', 'country_index'], inplace=True)

In [124]:
df4.drop(columns=['Unnamed: 0'], inplace=True)

In [125]:
df4 = df4.groupby(level=[0, 1]).mean()

In [126]:
print(df4.index)

MultiIndex([('2014-12-31 23:00',      'GB'),
            ('2015-01-01 00:00',      'AT'),
            ('2015-01-01 00:00',      'BE'),
            ('2015-01-01 00:00',      'CH'),
            ('2015-01-01 00:00',      'CZ'),
            ('2015-01-01 00:00',      'ES'),
            ('2015-01-01 00:00',      'FR'),
            ('2015-01-01 00:00',      'GB'),
            ('2015-01-01 00:00',      'HU'),
            ('2015-01-01 00:00', 'IT_CNOR'),
            ...
            ('2023-12-30 23:00',    'NO_3'),
            ('2023-12-31 00:00',      'BG'),
            ('2023-12-31 00:00',      'EE'),
            ('2023-12-31 00:00',      'FI'),
            ('2023-12-31 00:00',      'GR'),
            ('2023-12-31 00:00',      'LV'),
            ('2023-12-31 01:00',      'BG'),
            ('2023-12-31 01:00',      'EE'),
            ('2023-12-31 01:00',      'GR'),
            ('2023-12-31 01:00',      'LV')],
           names=['Date', 'country_index'], length=1976011)


In [127]:
df5 =pd.read_csv('oil_hydro_biomass_gas_nuclear_merged.csv')

In [128]:
df5.dtypes

Date                                   object
Country                                object
water_reservoirs_and_hydro_storage    float64
energy_price                          float64
biomass                               float64
gas                                   float64
nuclear                               float64
dtype: object

In [25]:
print(df5.index)

RangeIndex(start=0, stop=3575052, step=1)


In [129]:
df4 = df4.reset_index()

# Resetting the index for df5
df5 = df5.reset_index()

In [132]:
df4.rename(columns={'country_index': 'Country'}, inplace=True)

In [133]:
df4 = df4.set_index(['Date', 'Country'])
df5 = df5.set_index(['Date', 'Country'])

Now the merge of a single dataframe with all features 

In [134]:
print(df4.index.get_level_values('Date').dtype)
print(df5.index.get_level_values('Date').dtype)

period[H]
object


In [147]:
df5.drop(columns=['index'], inplace=True)

In [148]:
all_featuresdf = pd.merge(df4, df5, left_index=True, right_index=True, how='outer')

In [150]:
all_featuresdf

Solar  Wind Onshore  Wind Offshore  \
Date                Country                                       
2014-12-31 23:00    GB         0.0       4546.00         3165.0   
2015-01-01 00:00    AT         0.0        117.25            NaN   
                    BE         0.0        246.00          420.0   
                    CH         0.0          4.00            NaN   
                    CZ         0.0           NaN            NaN   
...                            ...           ...            ...   
2023-12-31 00:00:00 SE_2       NaN           NaN            NaN   
                    SE_3       NaN           NaN            NaN   
                    SE_4       NaN           NaN            NaN   
                    SI         NaN           NaN            NaN   
                    SK         NaN           NaN            NaN   

                             water_reservoirs_and_hydro_storage  energy_price  \
Date                Country                                                     
2014-12-31 23:00    GB                                      NaN           NaN   
2015-01-01 00:00    AT                                      NaN           NaN   
                    BE                                      NaN           NaN   
                    CH                                      NaN           NaN   
                    CZ                                      NaN           NaN   
...                                                         ...           ...   
2023-12-31 00:00:00 SE_2                                    NaN         44.87   
                    SE_3                                    NaN         44.87   
                    SE_4                                    NaN         44.87   
                    SI                                      NaN         33.30   
                    SK                                      NaN         35.14   

                             biomass  gas  nuclear  
Date                Country                         
2014-12-31 23:00    GB           NaN  NaN      NaN  
2015-01-01 00:00    AT           NaN  NaN      NaN  
                    BE           NaN  NaN      NaN  
                    CH           NaN  NaN      NaN  
                    CZ           NaN  NaN      NaN  
...                              ...  ...      ...  
2023-12-31 00:00:00 SE_2         NaN  NaN      NaN  
                    SE_3         NaN  NaN      NaN  
                    SE_4         NaN  NaN      NaN  
                    SI           NaN  NaN      NaN  
                    SK           NaN  NaN      NaN  

[5551063 rows x 8 columns]

In [155]:
all_featuresdf.tail(50)

Solar  Wind Onshore  \
Date                Country                                        
2023-12-30 23:00:00 RS                         NaN           NaN   
                    SE_1                       NaN           NaN   
                    SE_2                       NaN           NaN   
                    SE_3                       NaN           NaN   
                    SE_4                       NaN           NaN   
                    SI                         NaN           NaN   
                    SK                         NaN           NaN   
2023-12-31 00:00:00 BE                         NaN           NaN   
                    BG                         NaN           NaN   
                    CH                         NaN           NaN   
                    CZ                         NaN           NaN   
                    Combined_AT_DE_AT_LU       NaN           NaN   
                    Combined_DE_LU_DE_AT_LU    NaN           NaN   
                    DK_1                       NaN           NaN   
                    DK_2                       NaN           NaN   
                    EE                         NaN           NaN   
                    ES                         NaN           NaN   
                    FI                         NaN           NaN   
                    FR                         NaN           NaN   
                    GR                         NaN           NaN   
                    HR                         NaN           NaN   
                    HU                         NaN           NaN   
                    IT_CNOR                    NaN           NaN   
                    IT_CSUD                    NaN           NaN   
                    IT_NORD                    NaN           NaN   
                    IT_SACO_AC                 NaN           NaN   
                    IT_SACO_DC                 NaN           NaN   
                    IT_SARD                    NaN           NaN   
                    IT_SICI                    NaN           NaN   
                    IT_SUD                     NaN           NaN   
                    LT                         NaN           NaN   
                    LV                         NaN           NaN   
                    ME                         NaN           NaN   
                    MK                         NaN           NaN   
                    NL                         NaN           NaN   
                    NO_1                       NaN           NaN   
                    NO_2                       NaN           NaN   
                    NO_3                       NaN           NaN   
                    NO_4                       NaN           NaN   
                    NO_5                       NaN           NaN   
                    PL                         NaN           NaN   
                    PT                         NaN           NaN   
                    RO                         NaN           NaN   
                    RS                         NaN           NaN   
                    SE_1                       NaN           NaN   
                    SE_2                       NaN           NaN   
                    SE_3                       NaN           NaN   
                    SE_4                       NaN           NaN   
                    SI                         NaN           NaN   
                    SK                         NaN           NaN   

                                             Wind Offshore  \
Date                Country                                  
2023-12-30 23:00:00 RS                                 NaN   
                    SE_1                               NaN   
                    SE_2                               NaN   
                    SE_3                               NaN   
                    SE_4                               NaN   
                    SI                                 NaN   
            

In [156]:
all_featuresdf.to_csv('all_features_merged.csv')

In [158]:
df = pd.read_csv('all_features_merged.csv')

In [159]:
df

,Date,Country,Solar,Wind Onshore,Wind Offshore,water_reservoirs_and_hydro_storage,energy_price,biomass,gas,nuclear
0,2014-12-31 23:00,GB,0.0,4546.00,3165.0,NaN,NaN,NaN,NaN,NaN
1,2015-01-01 00:00,AT,0.0,117.25,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01 00:00,BE,0.0,246.00,420.0,NaN,NaN,NaN,NaN,NaN
3,2015-01-01 00:00,CH,0.0,4.00,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01 00:00,CZ,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5551058,2023-12-31 00:00:00,SE_2,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN
5551059,2023-12-31 00:00:00,SE_3,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN
5551060,2023-12-31 00:00:00,SE_4,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN
5551061,2023-12-31 00:00:00,SI,NaN,NaN,NaN,NaN,33.30,NaN,NaN,NaN
